In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split



In [2]:
# Load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to a range of 0 to 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data to fit the model
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [3]:
def create_model(depth=1, width=64, dropout_rate=0.2):
    model = Sequential()
    model.add(Dense(width, input_shape=(28*28,), activation='relu'))
    for _ in range(depth - 1):
        model.add(Dense(width, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [4]:
# Example of training the model with specific hyperparameters
model = create_model(depth=2, width=128, dropout_rate=0.3)

history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))




Epoch 1/10


1500/1500 [==============================] - 5s 3ms/step - loss: 0.3009 - accuracy: 0.9115 - val_loss: 0.1676 - val_accuracy: 0.9502
Epoch 2/10
1500/1500 [==============================] - 4s 2ms/step - loss: 0.1271 - accuracy: 0.9623 - val_loss: 0.1188 - val_accuracy: 0.9631
Epoch 3/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.0886 - accuracy: 0.9729 - val_loss: 0.0989 - val_accuracy: 0.9683
Epoch 4/10
1500/1500 [==============================] - 4s 2ms/step - loss: 0.0690 - accuracy: 0.9785 - val_loss: 0.0861 - val_accuracy: 0.9753
Epoch 5/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0572 - accuracy: 0.9819 - val_loss: 0.0807 - val_accuracy: 0.9773
Epoch 6/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0483 - accuracy: 0.9851 - val_loss: 0.0799 - val_accuracy: 0.9787
Epoch 7/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0419 - accuracy: 0.9860 - val_loss: 0.0827 - val_accur

In [6]:
import numpy as np

# Define the hyperparameter grid
depth_options = [1, 2, 3]
width_options = [64, 128, 256]
dropout_options = [0.2, 0.3, 0.4]

best_accuracy = 0
best_params = {}

# Perform grid search
for depth in depth_options:
    for width in width_options:
        for dropout_rate in dropout_options:
            model = create_model(depth=depth, width=width, dropout_rate=dropout_rate)
            history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val), verbose=0)
            val_accuracy = np.max(history.history['val_accuracy'])
            
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_params = {'depth': depth, 'width': width, 'dropout_rate': dropout_rate}

print(f"Best Validation Accuracy: {best_accuracy}")
print(f"Best Hyperparameters: {best_params}")


Best Validation Accuracy: 0.9816666841506958
Best Hyperparameters: {'depth': 2, 'width': 128, 'dropout_rate': 0.3}


In [8]:
# Train the best model
best_model = create_model(depth=best_params['depth'], width=best_params['width'], dropout_rate=best_params['dropout_rate'])
best_model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Evaluate on the test set
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/10
1500/1500 [==============================] - 9s 5ms/step - loss: 0.3045 - accuracy: 0.9082 - val_loss: 0.1432 - val_accuracy: 0.9579
Epoch 2/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1316 - accuracy: 0.9603 - val_loss: 0.1014 - val_accuracy: 0.9697
Epoch 3/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0944 - accuracy: 0.9708 - val_loss: 0.0967 - val_accuracy: 0.9708
Epoch 4/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0724 - accuracy: 0.9774 - val_loss: 0.0901 - val_accuracy: 0.9743
Epoch 5/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0590 - accuracy: 0.9813 - val_loss: 0.0817 - val_accuracy: 0.9760
Epoch 6/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0463 - accuracy: 0.9844 - val_loss: 0.0819 - val_accuracy: 0.9778
Epoch 7/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0428 - accuracy: 0.9862 - val_loss: 0.0873 - val_accuracy: